In [35]:
import os
import object_detection
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format
import wget
import torch

In [36]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [37]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [38]:
CUSTOM_MODEL_NAME = 'my_ssd_mobnet' 
PRETRAINED_MODEL_NAME = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz'
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'
LABEL_MAP_NAME = 'label_map.pbtxt'
MAIN_FOLDER_PATH = os.path.dirname(os.getcwd())
MAIN_FOLDER_PATH

'D:\\Code_school_nam3ki2\\TestModel'

In [39]:
paths = {
    'WORKSPACE_PATH': os.path.join(MAIN_FOLDER_PATH, 'Tensorflow', 'workspace'),
    'SCRIPTS_PATH': os.path.join(MAIN_FOLDER_PATH, 'Tensorflow','scripts'),
    'APIMODEL_PATH': os.path.join(MAIN_FOLDER_PATH, 'Tensorflow','models'),
    'ANNOTATION_PATH': os.path.join(MAIN_FOLDER_PATH, 'Tensorflow', 'workspace','annotations'),
    'IMAGE_PATH': os.path.join(MAIN_FOLDER_PATH, 'Tensorflow', 'workspace','images'),
    'MODEL_PATH': os.path.join(MAIN_FOLDER_PATH, 'Tensorflow', 'workspace','models'),
    'PRETRAINED_MODEL_PATH': os.path.join(MAIN_FOLDER_PATH,'Tensorflow', 'workspace','pre-trained-models'),
    'CHECKPOINT_PATH': os.path.join(MAIN_FOLDER_PATH,'Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME), 
    'EXPORTED_MODEL_PATH': os.path.join(MAIN_FOLDER_PATH,'Tensorflow', 'workspace','exported-models', CUSTOM_MODEL_NAME),
    'PROTOC_PATH':os.path.join(MAIN_FOLDER_PATH,'Tensorflow','protoc')
}
paths

{'WORKSPACE_PATH': 'D:\\Code_school_nam3ki2\\TestModel\\Tensorflow\\workspace',
 'SCRIPTS_PATH': 'D:\\Code_school_nam3ki2\\TestModel\\Tensorflow\\scripts',
 'APIMODEL_PATH': 'D:\\Code_school_nam3ki2\\TestModel\\Tensorflow\\models',
 'ANNOTATION_PATH': 'D:\\Code_school_nam3ki2\\TestModel\\Tensorflow\\workspace\\annotations',
 'IMAGE_PATH': 'D:\\Code_school_nam3ki2\\TestModel\\Tensorflow\\workspace\\images',
 'MODEL_PATH': 'D:\\Code_school_nam3ki2\\TestModel\\Tensorflow\\workspace\\models',
 'PRETRAINED_MODEL_PATH': 'D:\\Code_school_nam3ki2\\TestModel\\Tensorflow\\workspace\\pre-trained-models',
 'CHECKPOINT_PATH': 'D:\\Code_school_nam3ki2\\TestModel\\Tensorflow\\workspace\\models\\my_ssd_mobnet',
 'EXPORTED_MODEL_PATH': 'D:\\Code_school_nam3ki2\\TestModel\\Tensorflow\\workspace\\exported-models\\my_ssd_mobnet',
 'PROTOC_PATH': 'D:\\Code_school_nam3ki2\\TestModel\\Tensorflow\\protoc'}

In [40]:
files = {
    'PIPELINE_CONFIG':os.path.join(MAIN_FOLDER_PATH,'Tensorflow', 'workspace','models', CUSTOM_MODEL_NAME, 'pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], TF_RECORD_SCRIPT_NAME), 
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME)
}
files

{'PIPELINE_CONFIG': 'D:\\Code_school_nam3ki2\\TestModel\\Tensorflow\\workspace\\models\\my_ssd_mobnet\\pipeline.config',
 'TF_RECORD_SCRIPT': 'D:\\Code_school_nam3ki2\\TestModel\\Tensorflow\\scripts\\generate_tfrecord.py',
 'LABELMAP': 'D:\\Code_school_nam3ki2\\TestModel\\Tensorflow\\workspace\\annotations\\label_map.pbtxt'}

In [41]:
# Create Directory Structure
for path in paths.values():
    if not os.path.exists(path):
        os.makedirs(path)

In [42]:
# Create Label Map
labels = [
    {'name':'licence', 'id':1}
]   

In [43]:
train_img_path = r'D:\Code_school_nam3ki2\TestModel\Tensorflow\workspace\images\train'
test_img_path = r'D:\Code_school_nam3ki2\TestModel\Tensorflow\workspace\images\test'

In [ ]:
if not os.path.exists(files['TF_RECORD_SCRIPT']):
    os.system(f"git clone https://github.com/nicknochnack/GenerateTFRecord {paths['SCRIPTS_PATH']}")
os.system(f"python {files['TF_RECORD_SCRIPT']} -x {train_img_path} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'train.record')}")
os.system(f"python {files['TF_RECORD_SCRIPT']} -x {test_img_path} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'test.record')}")

In [45]:
#Copy model config từ file config của pretrain_model
command_copy_model_config = f"copy {os.path.join(paths['EXPORTED_MODEL_PATH'], 'pipeline.config')} {os.path.join(paths['CHECKPOINT_PATH'])}"

In [46]:
!{command_copy_model_config}

        1 file(s) copied.


In [47]:
# Đọc tệp pipeline.config và lấy ra tất cả các cấu hình.
configs = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])

In [48]:
# Custom lại model
configs['model'].ssd.num_classes = len(labels)
configs['train_config'].batch_size = 4
configs['train_config'].fine_tune_checkpoint = os.path.join(paths['EXPORTED_MODEL_PATH'], 'checkpoint', 'ckpt-0')
configs['train_config'].fine_tune_checkpoint_type = "detection"
configs['train_input_config'].label_map_path = files['LABELMAP']
configs['train_input_config'].tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'train.record')]
configs['eval_input_configs'][0].label_map_path = files['LABELMAP']
configs['eval_input_configs'][0].tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'test.record')]

In [49]:
# Lưu lại custom model vào file config
pipeline_config = config_util.create_pipeline_proto_from_configs(configs)
config_util.save_pipeline_config(pipeline_config, os.path.dirname(files['PIPELINE_CONFIG']))

INFO:tensorflow:Writing pipeline config file to D:\Code_school_nam3ki2\TestModel\Tensorflow\workspace\models\my_ssd_mobnet\pipeline.config


In [50]:
trainnig_check_dir = r"D:\Code_school_nam3ki2\TestModel\check"
trainnig_check_dir

'D:\\Code_school_nam3ki2\\TestModel\\check'

In [51]:
#Train model
TRAINING_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'model_main_tf2.py')
command = "python {} --model_dir={} --pipeline_config_path={} --num_train_steps=10000 --run_once={} --eval_training_data={}".format(
    TRAINING_SCRIPT, trainnig_check_dir, files['PIPELINE_CONFIG'], True, True)

In [52]:
command

'python D:\\Code_school_nam3ki2\\TestModel\\Tensorflow\\models\\research\\object_detection\\model_main_tf2.py --model_dir=D:\\Code_school_nam3ki2\\TestModel\\check --pipeline_config_path=D:\\Code_school_nam3ki2\\TestModel\\Tensorflow\\workspace\\models\\my_ssd_mobnet\\pipeline.config --num_train_steps=10000 --run_once=True --eval_training_data=True'

In [53]:
!{command}

C:\Users\minh3\.conda\envs\py310\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
C:\Users\minh3\.conda\envs\py310\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.12.0 and strictly below 2.15.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.10.1 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want